In [2]:
from datetime import datetime
import pandas as pd
import numpy as np

In [3]:
f = open("fvtt-log-Thu-Feb-17-2022.txt",encoding="utf8")
data = f.read()
data_list = data.split("---------------------------")
clean = []
characters = []
#df = pd.DataFrame(columns= ["Character", "Roll Result","Action","D20 Roll", "Date"])
try:
    for i, item in enumerate(data_list):
        temp_list = item.split("\n")
        temp_list = list(filter(None, temp_list))
        roll_result = temp_list[1]
        try:
            found_action = temp_list[2].find("<strong>")
        except:
            found_action = -1
        if roll_result.isnumeric() and found_action>-1:
            dateString = temp_list[0][1:temp_list[0].index(",")]
            date = datetime.strptime(dateString,'%m/%d/%Y')
            character = temp_list[0][temp_list[0].index("]")+2:]
            if character not in characters:
                characters.append(character)
            action = temp_list[2][temp_list[2].index("<strong>")+8:temp_list[2].index("</strong>")]
            d20roll = temp_list[-1][temp_list[-1].index("=")+1:temp_list[-1].index("=")+4].strip()
            clean.append([character,roll_result,action,d20roll,date])
        data_list[i] = temp_list
    df = pd.DataFrame(clean, columns= ["Character", "Roll Result","Action","D20 Roll", "Date"])
except Exception as e:
    print(dateString)
    print(date)
    print(data_list[i])
    print("error in line " + str(i))
    print(e)


In [ ]:
df

In [ ]:
data_list

In [6]:
import matplotlib.pyplot as plt
import statistics

In [ ]:
players = ["Lucius","Trujh Spellshot","Mr. Biscuit Oliva","Razonixx","Ylem"]
summary = []
sinceWhen = datetime(2022,2,10)
x_dice = []

for player in characters:
    historic = []
    x_dice = []
    rolls = pd.DataFrame(data ={"Rolls":[0]*20})
    rolls.index = rolls.index+1
    #xi = list(range(len(rolls.index)))
    newdf = df[df["Character"] == player]
    newdf = newdf[newdf["Date"] > sinceWhen]
    newdf["D20 Roll"] = pd.to_numeric(newdf["D20 Roll"])
    if len(newdf.index) > 0:
        plt.figure()
        counts = newdf['D20 Roll'].value_counts()
        counts = newdf['D20 Roll'].value_counts()
        counts = counts.to_frame()
        counts.index = pd.to_numeric(counts.index)
        counts = counts.sort_index()
        newRolls = pd.concat([rolls, counts],axis=1)
        newRolls = newRolls.fillna(value={"D20 Roll":0})
        newRolls['D20 Roll'].plot.bar()
        plt.xlabel('Roll')
        plt.title(player + " Mean: " + str(newdf['D20 Roll'].mean()))
        plt.savefig(player + ".jpg", dpi=72)
        summary.append([newdf['D20 Roll'].mean(),player,len(newdf.index)])   

        
summary.sort()

In [ ]:
summary.reverse()
summary